<a href="https://colab.research.google.com/github/yuho-ta/signate/blob/main/preddisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ライブラリのimport
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
train = pd.read_csv('/content/drive/MyDrive/signate/練習 健診データによる肝疾患判定/train.csv')
test = pd.read_csv('/content/drive/MyDrive/signate/練習 健診データによる肝疾患判定/test.csv')
sample=pd.read_csv('/content/drive/MyDrive/signate/練習 健診データによる肝疾患判定/sample_submit .csv',header=None)

In [2]:
train.dtypes
train.shape

(891, 12)

In [3]:
train.head(10)

,id,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease
0,0,60,Male,2.9,1.3,170.9,42.1,37.1,5.5,2.9,1.01,1
1,1,28,Female,0.7,0.1,158.8,26.0,23.9,6.4,3.7,1.36,0
2,2,60,Male,23.1,12.5,962.0,53.0,40.9,6.8,3.3,0.96,1
3,3,20,Male,1.0,0.5,415.9,33.9,39.0,7.0,3.8,1.31,0
4,4,44,Female,0.6,0.3,152.9,40.9,42.0,4.5,2.1,1.04,0
5,5,62,Male,11.1,5.7,699.0,64.0,100.1,7.4,3.3,0.64,1
6,6,32,Male,12.4,6.0,514.9,48.1,92.1,6.5,2.5,0.81,1
7,7,37,Female,0.8,0.1,152.0,89.9,20.9,7.0,4.3,1.43,0
8,8,41,Male,0.9,0.2,114.0,20.9,22.9,7.0,3.1,1.04,0
9,9,14,Male,1.7,0.6,268.8,58.0,45.1,6.7,3.9,1.21,1


In [4]:
train["Gender"] = train["Gender"].apply(lambda x: 1 if x=="Male" else 0)
train["AG_ratio"].fillna(train["Alb"] / (train["TP"] - train["Alb"]), inplace=True)
y=train['disease']
train=train.drop(["id","disease","Gender"], axis=1)
test=test.drop(["id","Gender"],axis=1)
all_data = pd.concat((train, test)).reset_index(drop=True)

In [5]:
bins_T_Bil = [0, 0.5, 1.0, 100]
X_cut_T_Bil, bin_indice_T_Bil = pd.cut(all_data["T_Bil"], bins=bins_T_Bil, retbins=True, labels=False)
X_dummies_T_Bil = pd.get_dummies(X_cut_T_Bil, prefix=X_cut_T_Bil.name)
X_binned_T_Bil = pd.concat([all_data, X_dummies_T_Bil], axis=1)

In [6]:
bins_D_Bil = [0, 0.5, 1,1.5, 2]
X_cut_D_Bil, bin_indice_D_Bil = pd.cut(X_binned_T_Bil["D_Bil"], bins=bins_D_Bil, retbins=True, labels=False)
X_dummies_D_Bil = pd.get_dummies(X_cut_D_Bil, prefix=X_cut_D_Bil.name)
X_binned_D_Bil = pd.concat([X_binned_T_Bil, X_dummies_D_Bil], axis=1)
X_binned_T_Bil=X_binned_T_Bil.drop(["D_Bil","T_Bil"],axis=1)

In [7]:
# 多項式・交互作用特徴量
polynomial = PolynomialFeatures(degree=2, include_bias=False)
polynomial_arr = polynomial.fit_transform(X_binned_T_Bil)
X_polynomial = pd.DataFrame(polynomial_arr, columns=["poly" + str(x) for x in range(polynomial_arr.shape[1])])

In [8]:
# 組み込み法のモデル、閾値の指定
fs_model = LogisticRegression(penalty='l1', solver='liblinear', random_state=0)
fs_threshold = "mean"
# 組み込み法モデルの初期化
selector = SelectFromModel(fs_model, threshold=fs_threshold)

train_polynomial =  X_polynomial.head(891)
test_polynomial = X_polynomial.iloc[891:, :]
# 特徴量選択の実行
selector.fit(train_polynomial, y)
mask = selector.get_support()

# 選択された特徴量だけのサンプル取得
train_polynomial_masked = train_polynomial.loc[:, mask]
test_polynomial_masked = test_polynomial.loc[:, mask]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [9]:
#モデルの学習・予測
model = LogisticRegression()
model.fit(train_polynomial_masked, y)
y_pred = model.predict(test_polynomial_masked)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
y_pred_series = pd.Series(y_pred)
result_df = pd.concat([sample.iloc[:, 0],y_pred_series], axis=1)
result_df.to_csv('/content/drive/MyDrive/signate/練習 健診データによる肝疾患判定/submit.csv', header=False, index=False)